<a href="https://colab.research.google.com/github/Zelechos/Generador-Css/blob/main/model/GeneradorCSS_v0.2.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Generador de CSS [v0.2.0]**

---



# Clonamos nuestro repositorio en donde se encuentra nuestro dataset con este comando

In [1]:
!git clone https://github.com/Zelechos/Generador-Css.git

Cloning into 'Generador-Css'...
remote: Enumerating objects: 708, done.
remote: Counting objects: 100% (307/307), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 708 (delta 176), reused 275 (delta 151), pack-reused 401
Receiving objects: 100% (708/708), 52.79 MiB | 28.53 MiB/s, done.
Resolving deltas: 100% (362/362), done.


### en este caso no es necesario ya que tenemos el dataset en github por ende tenemos que trabajar con el dataset que esta en formato json

Intalamos la libreria trasformers datasets

In [24]:
! pip install transformers datasets

### Cargamos nuestro DATASET


In [25]:
from datasets import load_dataset


data_files = {
    "train": "/content/Generador-Css/dataset/STYLES.json",
    "validation": "/content/Generador-Css/dataset/TEST.json",
    "test": "/content/Generador-Css/dataset/TEST.json"
}

dataset = load_dataset("json", data_files=data_files, field='styles')

# Analisamos el Dataset

In [26]:
print(dataset.data)

{'train': MemoryMappedTable
selector: string
style: string
----
selector: [["body","div.drop-container","div.drop-container","div.drop-container","div.drop-container",...,".image-rotate",".image-rotate",".image-rotate",".image-rotate",".image-rotate"],[".image-rotate",".image-rotate",".image-rotate",".image-rotate",".image-rotate",...,".promos",".promos",".promos",".promos",".promos"]]
style: [["body { overflow:hidden ; background:rgb(25,35,125) } ","div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ","div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ","div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ","div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ",...,".image-rotate { overflow:hidden ; margin:

### Cargando nuestro modelo Pre-entrenado GPT-2 XL

In [27]:
from transformers import GPT2Tokenizer, TFGPT2Model

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = TFGPT2Model.from_pretrained('gpt2-xl')

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)


ResourceExhaustedError: {{function_node __wrapped__StatelessTruncatedNormalV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[50257,1600] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessTruncatedNormalV2] name: 

In [6]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2-xl')
set_seed(42)
generator("The man worked as a", max_length=10, num_return_sequences=5)

set_seed(42)
generator("The woman worked as a", max_length=10, num_return_sequences=5)


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The woman worked as a supervisor for the company.'},
 {'generated_text': "The woman worked as a volunteer at the Children's"},
 {'generated_text': 'The woman worked as a waitress at a New Jersey'},
 {'generated_text': 'The woman worked as a lab technician but quit to'},
 {'generated_text': 'The woman worked as a prostitute, according to reports'}]

Analisis del Modelo GTP-2 XL

**Experimento 001**
- Cambiamos el parametro max_length a 50
que dadas las observaciones define la longitud de palabras maximas que puede tener la sentencia a generar
- El parametro num_return_sequences = 1

In [ ]:
set_seed(42)
generator("Alan Turing was a", max_length=50, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Alan Turing was a brilliant mathematician, but you can\'t beat a human being in a chess match with any of the chess engines out there. So for computer chess, he said, "You are only ever going to beat a human being in a game'}]

**Experimento 002**
- Cambiamos el parametro max_length a 2
que dadas las observaciones define la longitud de palabras maximas que puede tener la sentencia a generar
- Tambien Cambiamos el parametro num_return_sequences a 3
que dadas las observaciones es la cantidad de sentencias que genera el modelo considerando la longitud de palabras antes definida

*Se observo que al poner el parametro de max_length = 2 presenta un error por la longitud del input*

In [ ]:
generator("a Transformer in AI is", max_length=2, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ValueError: Input length of input_ids is 2, but `max_length` is set to 2. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

**Experimento 003**
- Cambiamos el parametro max_length a 7
que dadas las observaciones define la longitud de palabras maximas que puede tener la sentencia a generar incluyendo el input
- Tambien Cambiamos el parametro num_return_sequences a 3
que dadas las observaciones es la cantidad de sentencias que genera el modelo considerando la longitud de palabras antes definida

Se observo que con el parametro max_length = 2 hay una error dado que el parametro max_length debe superar en longitud de input_ids es N entonces el parametro max_length siempre tiene que ser almenos N + 1 o mayor que.

In [ ]:
generator("a Transformer in AI is", max_length=10, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'a Transformer in AI is the same as that'},
 {'generated_text': 'a Transformer in AI is a powerful engine ('},
 {'generated_text': 'a Transformer in AI is more than worth it'}]

**Experimento 004**
- Definimos el parametro de la semilla en 30 para observar que sucede


In [ ]:
set_seed(30)
generator("a Artificial Neural Network is", max_length=15, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'a Artificial Neural Network is a machine learning algorithm that combines a number of techniques'},
 {'generated_text': 'a Artificial Neural Network is a program capable of learning and recognizing symbols and patterns'},
 {'generated_text': 'a Artificial Neural Network is trained on a large number of digitized music examples'}]

# Aplicar Transfer Learning de GPT-2 XL a GeneradorCSS

In [23]:
from transformers import GPT2Tokenizer, TFGPT2Model

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = TFGPT2Model.from_pretrained('gpt2-xl')

# Selecciona un token existente como token de relleno
tokenizer.pad_token = tokenizer.eos_token

# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

Seleccionando un token de relleno existente

In [3]:
# Selecciona un token existente como token de relleno
tokenizer.pad_token = tokenizer.eos_token


Creamos la funcionr para tokenizar el dataset

In [11]:
def tokenize_function(examples):
  return tokenizer(examples["style"], padding="max_length", truncation=True)

Tokenizamos nuestro dataset

In [15]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(tokenized_datasets['train'])

Dataset({
    features: ['selector', 'style', 'input_ids', 'attention_mask'],
    num_rows: 1337
})


Si lo desea, puede crear un subconjunto más pequeño del conjunto de datos completo para realizar ajustes y reducir el tiempo necesario:

In [16]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Dataset({
    features: ['selector', 'style', 'input_ids', 'attention_mask'],
    num_rows: 1000
})


In [18]:
import numpy as np
from transformers import AutoTokenizer

labels = np.array(tokenized_datasets['train']['selector'])  # Label is already an array of 0 and 1

In [21]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam


In [22]:

# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5), loss='sparse_categorical_crossentropy')

model.fit(tokenized_datasets['train'], labels)

ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x7ec505f6a830>